I'm using the conda 3.12 environment when it's running locally.  I had to manually install the following.

```sh
conda env list
conda activate py3.12
conda install -c conda-forge python-dotenv
conda install pillow
pip install openai==1.1.1
conda install requests
```

However, when using Github codespace, conda is not working, but venv is, so the above is not necessary when working in codespace.


In [ ]:
!pip show openai


In [ ]:
import os
import openai
from openai import OpenAI

api_key=os.environ["OPENAI_API_KEY"]
client = OpenAI(api_key=api_key)

DALL-E 3

In [ ]:
PROMPT = "a room full of dogs all meditating in a circle"

In [ ]:
import openai
from openai import OpenAI
client = OpenAI(api_key=api_key)

response = client.images.generate(
  model="dall-e-3",
  prompt= PROMPT,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
image_url

In [ ]:
import requests
from PIL import Image
from io import BytesIO
from IPython.display import display

def display_and_save_image_from_url(url, scale_percent=100, save_name='image.png'):
    # Send a GET request to the specified URL to retrieve the image
    response = requests.get(url)
    # Open the image
    img = Image.open(BytesIO(response.content))

    # Calculate the new size, as a percentage of the original size
    if scale_percent != 100:
        width, height = img.size
        new_width = int(width * scale_percent / 100)
        new_height = int(height * scale_percent / 100)
        img = img.resize((new_width, new_height))

    # Save the image locally with the given name
    img.save(save_name)

    # Display the image in the notebook
    display(img)

# Call the function with the URL, the scale percentage, and the save name you want
display_and_save_image_from_url(image_url, scale_percent=50, save_name='../tmp/meditating_dogs.png')

GPT Voice


In [ ]:
import base64
import requests

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../tmp/meditating_dogs.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}"
}

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Describe what's in this image in detail as a story?"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

In [ ]:
story = response.json()['choices'][0]['message']['content']

print(story)

## Text To Speech


In [ ]:
from IPython.display import Audio
from pathlib import Path

response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input=story
)

# Define the path where you want to save the file
speech_file_path = Path('../tmp/story.mp3')

# Save the response content (binary content of the mp3 file) to the path
with open(speech_file_path, 'wb') as file:
    file.write(response.content)

# Play the audio file
Audio(speech_file_path, autoplay=True)
